In [3]:
%pip install pillow
%pip install pyyaml
%pip install sqlalchemy
%pip install pymysql
%pip install openai
%pip install imageio
%pip install ffmpeg
%pip install imageio-ffmpeg
%pip install moviepy
%pip install soundfile
%pip install opencv-python
%pip install fastapi
%pip install sentence_transformers=3.1.1

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'sentence_transformers=3.1.1': Expected end or semicolon (after name and no valid version specifier)
    sentence_transformers=3.1.1
                         ^
Hint: = is not a valid operator. Did you mean == ?


In [7]:
import json
import os
import sys

# local_creat.py가 있는 경로를 sys.path에 추가
module_path = os.path.abspath(os.path.join('..', 'article_creator'))
if module_path not in sys.path:
    sys.path.append(module_path)

from embedd import embedd
import numpy as np
from PIL import Image  # Pillow를 사용하여 이미지를 처리
from local_creat import create_video_and_save
from datetime import datetime

# JSON 형식의 대사 데이터
dialogue_data = [
  {
    "scene": 1,
    "description": "A job seeker is looking at a job posting board at the 2024 Aviation Industry Job Fair held at Incheon International Airport. The event draws attention from many young job seekers who are navigating a tough job market.",
    "dialogue_en": "Job fairs like this provide hope for many looking to break into competitive industries. 'I hope to find a good opportunity here,' says one attendee.",
    "dialogue_ko": "'이런 채용박람회가 경쟁이 치열한 산업에 진입할 수 있는 기회를 제공해줘요.' 한 구직자는 '좋은 기회를 찾고 싶다'고 말합니다."
  },
  {
    "scene": 2,
    "description": "Data released by the National Statistical Office shows a steady increase in employment figures for the third month in a row. Despite overall growth, industries such as retail and construction are seeing significant job losses.",
    "dialogue_en": "'The job market is showing signs of recovery,' says an official, 'but some sectors like retail and construction are still facing challenges.' Employment in these sectors has been decreasing for several months.",
    "dialogue_ko": "'일자리 시장이 회복되는 신호를 보이고 있지만,' 한 관계자는 '도소매업과 건설업 같은 일부 부문은 여전히 어려움을 겪고 있다'고 말합니다. 이들 업종의 고용은 몇 달째 감소세입니다."
  },
  {
    "scene": 3,
    "description": "The report highlights how the information and communication sectors, as well as professional scientific and technical services, have seen notable employment growth. These industries are leading the job recovery in the country.",
    "dialogue_en": "'Sectors like IT and science are thriving,' says an economist. 'They are driving the country's economic growth and offering new job opportunities.'",
    "dialogue_ko": "한 경제학자는 'IT와 과학 같은 분야가 성장하고 있다'며 '이들이 국가 경제 성장을 이끌며 새로운 일자리 기회를 제공하고 있다'고 말합니다."
  },
  {
    "scene": 4,
    "description": "The job market for younger workers remains a concern, with a significant decline in employment among those under 30. In contrast, employment for those over 60 continues to rise, reflecting the aging population.",
    "dialogue_en": "'The younger generation is struggling to find stable jobs,' an expert notes. 'At the same time, older workers are increasingly staying in the workforce, driving employment gains in the senior demographic.'",
    "dialogue_ko": "전문가는 '젊은 세대는 안정적인 일자리를 찾는 데 어려움을 겪고 있다'며 '동시에 고령층은 계속 일자리에 남아 있어 이들의 고용 증가를 이끌고 있다'고 설명합니다."
  },
  {
    "scene": 5,
    "description": "Despite challenges, South Korea's overall employment rate reached its highest level for September since 1982, with the OECD employment rate also setting a record. This shows resilience in the job market despite the challenges in certain sectors.",
    "dialogue_en": "'The employment rate reaching such high levels is encouraging,' says a government spokesperson. 'It indicates that the overall economy is strong, even if some industries are lagging.'",
    "dialogue_ko": "정부 대변인은 '고용률이 이처럼 높은 수준에 도달한 것은 고무적이다'라며 '일부 산업이 뒤처지더라도 전체 경제가 강하다는 것을 보여준다'고 말합니다."
  },
  {
    "scene": 6,
    "description": "Unemployment rates continue to decline, hitting record lows for September since 1999. However, the number of people not participating in the labor market, such as retirees and students, has slightly increased.",
    "dialogue_en": "'It's a promising sign that unemployment is falling,' says an economist. 'But we also need to consider the growing number of people choosing not to participate in the labor market.'",
    "dialogue_ko": "경제학자는 '실업률이 감소하는 것은 긍정적인 신호'라며 '그러나 노동 시장에 참여하지 않는 인구가 증가하는 것도 고려해야 한다'고 지적합니다."
  }
]

def convert_datetime_to_mysql_format(iso_datetime_str):
    # ISO 8601 문자열을 파싱하여 datetime 객체로 변환
    dt = datetime.fromisoformat(iso_datetime_str)
    # MySQL DATETIME 형식 (YYYY-MM-DD HH:MM:SS)으로 변환하여 반환
    return dt.strftime('%Y-%m-%d %H:%M:%S')

# 이미지 파일을 numpy 배열로 변환하는 함수
def load_image_as_array(image_path):
    img = Image.open(image_path)
    return np.array(img)

# 기사 데이터를 읽고 영상을 생성하는 메서드
def process_article_for_video(article_dir):
    # 기사 JSON 파일 경로
    article_json_path = os.path.join(article_dir, 'source-2024-10-16-1.json')
    
    # 이미지 파일 경로
    images_dir = os.path.join(article_dir, 'images')
    image_files = sorted([os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith('.png')])
    
    # 이미지 번호에 맞는 대사 매칭
    dialogues = []  # 대사 문자열 배열
    image_arrays = []  # 이미지 numpy 배열
    for image_file in image_files:
        # 이미지 파일명에서 숫자 추출 (예: scene-1.png -> 1)
        image_num = int(os.path.basename(image_file).split('-')[1].split('.')[0])
        
        # 해당 scene에 맞는 대사 찾기
        dialogue = next((item for item in dialogue_data if item["scene"] == image_num), None)
        
        if dialogue:
            # dialogue_ko를 사용하여 대사 문자열 배열에 추가
            dialogues.append(dialogue['dialogue_ko'])
        else:
            dialogues.append('')  # 해당 scene이 없으면 공백
        
        # 이미지를 numpy 배열로 변환하여 리스트에 추가
        image_arrays.append(load_image_as_array(image_file))

    # 기사 데이터 로드 (json 파일은 딕셔너리 형태이므로 리스트 인덱스를 제거)
    with open(article_json_path, 'r', encoding='utf-8') as f:
        article_data = json.load(f)  # 딕셔너리 형태로 데이터 로드
        
    # source_created_at 날짜를 MySQL 형식으로 변환
    article_data['source_created_at'] = convert_datetime_to_mysql_format(article_data['source_created_at'])
    
    # 함수 호출 (이미지 배열, 대화, 기사 데이터 전달)
    create_video_and_save(
        images=image_arrays,  # 이미지 배열 전달
        dialogues=dialogues,  # 대사 문자열 배열 전달
        title=article_data['title'],
        source_url=article_data['source_url'],
        category=article_data['category'],
        source_created_at=article_data['source_created_at'],
        press_id=article_data['press_id'],
        content=article_data['content'],
        press_name=article_data['press_name']
    )

# 기사 폴더 경로 예시
article_dir = './joongang/economy/2024-10-16-1'
process_article_for_video(article_dir)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


INFO: 2024-10-16 10:04:09,871 - es - save article(id=134, title=9월 취업자 14만4000명↑…건설업, 역대 최대폭 감소)
INFO: 2024-10-16 10:04:09,871 - es - 실행시간: 0.440201997756958
